In [250]:
import json
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import chain
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

### Read data

In [306]:
# Load the pickled list
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/zeroshot/results/comics35_zs_pg_p2_Meta-Llama-3.1-8B-Instruct-bnb-4bit/results.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [307]:
results

{'grounds': ["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
  "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
  "[['joy'], ['joy'], ['anger'], ['anger']]",
  "[['fear', 'surprise']]",
  "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
  "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
  "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
  "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
  "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sad

In [308]:
#all_labels = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]

In [309]:
grounds = results['grounds']
predictions = results['predictions']

In [310]:
len(grounds), len(predictions)

(874, 874)

In [311]:
grounds

["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
 "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
 "[['joy'], ['joy'], ['anger'], ['anger']]",
 "[['fear', 'surprise']]",
 "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
 "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
 "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
 "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
 "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sadness'], ['anger']]",

In [312]:
predictions

['{"page_utterance_emotions": [["anger", "disgust"], ["anger", "disgust"], ["fear", "joy"], ["sadness", "fear"], ["anger", "joy"], ["fear", "sadness"], ["fear", "joy"], ["anger", "surprise"], ["anger", "surprise"], ["anger", "surprise", "joy"]]}',
 '{"page_utterance_emotions": [["anger", "disgust"], ["surprise", "joy"], ["anger", "fear"], ["anger", "fear", "surprise"], ["anger", "joy"], ["disgust", "anger"], ["neutral"], ["anger", "fear"], ["anger", "fear", "joy"]]}',
 '{"page_utterance_emotions": [["anger", "anger"], ["anger"], ["anger", "fear"], ["anger", "joy"]]}',
 '{"page_utterance_emotions": [["anger"]]}',
 '{"page_utterance_emotions": [["joy", "anger"], ["joy", "anger"], ["anger", "fear"], ["anger", "fear"], ["surprise", "joy"], ["anger", "fear"]]}',
 '{"page_utterance_emotions": [["anger", "surprise"], ["fear", "surprise"], ["anger"], ["neutral"], ["anger", "fear"], ["anger", "fear"], ["fear", "surprise"], ["fear", "surprise"], ["joy", "neutral"], ["anger", "fear", "joy"]]}',
 

In [258]:
#grounds = [json.loads(x)["page_utterance_emotions"] for x in grounds]

In [259]:
import re

In [260]:
processed_preds = []
bad_idx = []

for i, input_string in enumerate(predictions):
    
    json_content = re.search(r'\{.*\}', input_string, re.DOTALL)
    
    if json_content:
        json_string = json_content.group()
        processed_preds.append(json_string)
    else:    
        bad_idx.append(i)
        print(i)
        

28
29
39
43
50
66
70
74
88
102
112
113
124
129
130
136
139
152
154
162
168
178
179
187
197
198
200
211
220
243
252
253
256
273
274
281
283
286
289
297
301
305
314
316
326
328
331
336
339
341
354
358
369
390
423
429
438
447
460
473
480
486
489
492
496
497
502
521
538
542
545
552
563
566
572
579
580
581
585
586
588
591
594
604
605
607
610
615
621
623
625
627
636
643
669
671
677
687
711
714
718
727
728
745
758
761
768
769
773
774
783
788
800
803
804
820
825
832
835
850
864
872


In [305]:
predictions[66]

['joy']

In [314]:
bad_idx = []
processed_preds = []

for i,x in enumerate(predictions):
    try:
        processed_preds.append(json.loads(x)["page_utterance_emotions"])
    except:
        print(i)
        bad_idx.append(i)

9
33
35
40
45
47
50
52
57
80
83
85
93
106
107
111
116
118
128
141
142
143
144
145
156
173
183
193
197
200
202
211
214
216
221
231
234
238
239
249
251
254
273
276
280
283
285
290
295
301
329
330
336
347
354
361
363
376
377
380
388
402
408
410
414
417
435
446
465
467
478
484
492
505
528
534
544
550
557
566
578
580
584
602
612
627
633
636
645
646
648
661
664
668
670
686
687
690
710
715
726
750
753
760
767
775
776
780
783
786
790
808
810
813
819
834
839
843
849
860
861
863
864
865


In [318]:
predictions[35]

'{"page_utterance_emotions": [["neutral"],["anger"],["neutral"],["sadness"],["sadness"],["anger"],["sadness"],["neutral"],["neutral"],["neutral"]]}\n\nor\n\n{"page_utterance_emotions": [["neutral"],["anger"],["neutral"],["sadness"],["sadness"],["anger"],["sadness"],["neutral"],["neutral"],["neutral"]]}'

In [315]:
len(grounds), len(processed_preds)

(874, 750)

In [319]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    #del predictions[idx]

In [320]:
len(grounds), len(processed_preds)

(750, 750)

In [ ]:
#predictions = [json.loads(x)["page_utterance_emotions"] for x in processed_preds]

In [ ]:
#len(grounds), len(predictions)

(752, 752)

In [321]:
grounds

["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
 "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
 "[['joy'], ['joy'], ['anger'], ['anger']]",
 "[['fear', 'surprise']]",
 "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
 "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
 "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
 "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
 "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sadness'], ['anger']]",

In [322]:
import ast

In [323]:
grounds = [ast.literal_eval(x) for x in grounds]

In [324]:
grounds

[[['anger'],
  ['anger'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['sadness'],
  ['anger'],
  ['surprise'],
  ['surprise'],
  ['fear', 'surprise']],
 [['fear'],
  ['anger'],
  ['surprise'],
  ['anger'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger']],
 [['joy'], ['joy'], ['anger'], ['anger']],
 [['fear', 'surprise']],
 [['anger'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['anger', 'joy'],
  ['anger']],
 [['sadness'],
  ['fear'],
  ['joy'],
  ['neutral'],
  ['surprise'],
  ['fear'],
  ['surprise'],
  ['joy'],
  ['surprise'],
  ['sadness', 'surprise']],
 [['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['anger', 'surprise'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['surprise'],
  ['surprise'],
  ['anger'],
  ['anger']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['anger', 'fear', 'surprise'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['anger'],
  ['anger'],
  ['surpris

In [325]:
processed_preds

[[['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'joy'],
  ['sadness', 'fear'],
  ['anger', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'joy'],
  ['anger', 'surprise'],
  ['anger', 'surprise'],
  ['anger', 'surprise', 'joy']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['anger', 'joy'],
  ['disgust', 'anger'],
  ['neutral'],
  ['anger', 'fear'],
  ['anger', 'fear', 'joy']],
 [['anger', 'anger'], ['anger'], ['anger', 'fear'], ['anger', 'joy']],
 [['anger']],
 [['joy', 'anger'],
  ['joy', 'anger'],
  ['anger', 'fear'],
  ['anger', 'fear'],
  ['surprise', 'joy'],
  ['anger', 'fear']],
 [['anger', 'surprise'],
  ['fear', 'surprise'],
  ['anger'],
  ['neutral'],
  ['anger', 'fear'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['fear', 'surprise'],
  ['joy', 'neutral'],
  ['anger', 'fear', 'joy']],
 [['joy', 'surprise'],
  ['anger', 'disgust'],
  ['neutral'],
  ['surprise'],
  ['joy', 'neutral']],
 [['surprise', 'joy'],
 

In [328]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, processed_preds)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

7 12 13
8 14 16
11 13 14
19 9 10
22 12 15
25 14 15
38 13 14
41 11 12
51 8 7
54 6 4
57 8 9
63 13 12
66 18 19
72 13 16
75 10 11
76 10 13
82 8 9
86 11 12
89 21 24
91 20 19
92 15 16
93 11 12
94 14 20
96 9 10
98 23 22
99 13 14
100 13 15
102 11 12
106 13 15
111 9 10
112 12 13
113 12 14
122 3 4
123 9 10
135 13 14
139 6 7
143 8 9
145 11 14
148 2 3
156 9 10
165 13 14
169 18 17
174 22 21
186 6 5
191 11 12
196 17 16
197 13 14
198 9 10
206 13 14
208 16 18
209 15 16
210 14 15
214 6 7
217 8 9
230 19 22
231 23 25
232 13 15
238 9 10
239 14 15
240 11 12
243 17 18
244 15 14
245 11 12
250 9 12
256 13 16
259 11 15
264 7 8
266 9 10
268 13 12
270 19 20
272 11 15
273 14 13
275 14 16
276 12 14
277 10 11
279 2 4
282 5 10
283 1 3
286 4 5
287 2 3
288 4 10
293 9 10
297 14 16
301 9 10
302 1 2
304 11 12
305 16 17
308 11 16
309 11 12
311 15 17
316 6 7
324 8 10
325 7 9
333 8 9
339 9 10
340 7 6
341 14 18
344 13 15
347 15 17
348 10 11
349 11 12
351 11 12
352 11 12
359 9 10
362 6 5
372 11 12
378 9 10
379 8 10
380 9 11
3

In [329]:
len(bad_idx)

205

In [330]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    del processed_preds[idx]

In [331]:
len(grounds), len(processed_preds)

(545, 545)

In [332]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in processed_preds for item in sublist]

In [333]:
#grounds = [[item for inner in outer for item in inner] for outer in grounds]
#predictions = [[item for inner in outer for item in inner] for outer in predictions]

In [334]:
len(grounds), len(predictions)

(3685, 3685)

In [335]:
grounds

[['anger'],
 ['anger'],
 ['fear'],
 ['fear'],
 ['fear', 'sadness'],
 ['sadness'],
 ['anger'],
 ['surprise'],
 ['surprise'],
 ['fear', 'surprise'],
 ['fear'],
 ['anger'],
 ['surprise'],
 ['anger'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['fear', 'surprise'],
 ['anger'],
 ['anger'],
 ['fear'],
 ['fear', 'surprise'],
 ['anger', 'joy'],
 ['anger'],
 ['sadness'],
 ['fear'],
 ['joy'],
 ['neutral'],
 ['surprise'],
 ['fear'],
 ['surprise'],
 ['joy'],
 ['surprise'],
 ['sadness', 'surprise'],
 ['surprise'],
 ['fear', 'sadness'],
 ['surprise'],
 ['surprise'],
 ['joy'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['fear'],
 ['sadness'],
 ['sadness'],
 ['sadness'],
 ['fear', 'sadness'],
 ['fear'],
 ['fear'],
 ['sadness'],
 ['anger', 'surprise'],
 ['anger', 'joy'],
 ['anger'],
 ['anger'],
 ['fear'],
 ['surprise', 'joy'],
 ['fear'],
 ['surprise'],
 ['fear'],
 ['fear', 'surprise'],
 ['fear'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['jo

In [336]:
predictions

[['anger', 'disgust'],
 ['anger', 'disgust'],
 ['fear', 'joy'],
 ['sadness', 'fear'],
 ['anger', 'joy'],
 ['fear', 'sadness'],
 ['fear', 'joy'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['anger', 'surprise', 'joy'],
 ['anger', 'disgust'],
 ['surprise', 'joy'],
 ['anger', 'fear'],
 ['anger', 'fear', 'surprise'],
 ['anger', 'joy'],
 ['disgust', 'anger'],
 ['neutral'],
 ['anger', 'fear'],
 ['anger', 'fear', 'joy'],
 ['anger', 'anger'],
 ['anger'],
 ['anger', 'fear'],
 ['anger', 'joy'],
 ['anger'],
 ['joy', 'anger'],
 ['joy', 'anger'],
 ['anger', 'fear'],
 ['anger', 'fear'],
 ['surprise', 'joy'],
 ['anger', 'fear'],
 ['anger', 'surprise'],
 ['fear', 'surprise'],
 ['anger'],
 ['neutral'],
 ['anger', 'fear'],
 ['anger', 'fear'],
 ['fear', 'surprise'],
 ['fear', 'surprise'],
 ['joy', 'neutral'],
 ['anger', 'fear', 'joy'],
 ['joy', 'surprise'],
 ['anger', 'disgust'],
 ['neutral'],
 ['surprise'],
 ['joy', 'neutral'],
 ['surprise', 'joy'],
 ['neutral'],
 ['fear', 'joy'],
 ['joy'],
 ['joy

In [337]:
predictions = [
    sublist[0] if isinstance(sublist, list) and len(sublist) == 1 and isinstance(sublist[0], list) else sublist
    for sublist in predictions
]

In [338]:
predictions

[['anger', 'disgust'],
 ['anger', 'disgust'],
 ['fear', 'joy'],
 ['sadness', 'fear'],
 ['anger', 'joy'],
 ['fear', 'sadness'],
 ['fear', 'joy'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['anger', 'surprise', 'joy'],
 ['anger', 'disgust'],
 ['surprise', 'joy'],
 ['anger', 'fear'],
 ['anger', 'fear', 'surprise'],
 ['anger', 'joy'],
 ['disgust', 'anger'],
 ['neutral'],
 ['anger', 'fear'],
 ['anger', 'fear', 'joy'],
 ['anger', 'anger'],
 ['anger'],
 ['anger', 'fear'],
 ['anger', 'joy'],
 ['anger'],
 ['joy', 'anger'],
 ['joy', 'anger'],
 ['anger', 'fear'],
 ['anger', 'fear'],
 ['surprise', 'joy'],
 ['anger', 'fear'],
 ['anger', 'surprise'],
 ['fear', 'surprise'],
 ['anger'],
 ['neutral'],
 ['anger', 'fear'],
 ['anger', 'fear'],
 ['fear', 'surprise'],
 ['fear', 'surprise'],
 ['joy', 'neutral'],
 ['anger', 'fear', 'joy'],
 ['joy', 'surprise'],
 ['anger', 'disgust'],
 ['neutral'],
 ['surprise'],
 ['joy', 'neutral'],
 ['surprise', 'joy'],
 ['neutral'],
 ['fear', 'joy'],
 ['joy'],
 ['joy

In [339]:
predictions = [
    [item for sublist in lst for item in sublist] if all(isinstance(i, list) for i in lst) else lst
    for lst in predictions
]

In [340]:
predictions

[['anger', 'disgust'],
 ['anger', 'disgust'],
 ['fear', 'joy'],
 ['sadness', 'fear'],
 ['anger', 'joy'],
 ['fear', 'sadness'],
 ['fear', 'joy'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['anger', 'surprise', 'joy'],
 ['anger', 'disgust'],
 ['surprise', 'joy'],
 ['anger', 'fear'],
 ['anger', 'fear', 'surprise'],
 ['anger', 'joy'],
 ['disgust', 'anger'],
 ['neutral'],
 ['anger', 'fear'],
 ['anger', 'fear', 'joy'],
 ['anger', 'anger'],
 ['anger'],
 ['anger', 'fear'],
 ['anger', 'joy'],
 ['anger'],
 ['joy', 'anger'],
 ['joy', 'anger'],
 ['anger', 'fear'],
 ['anger', 'fear'],
 ['surprise', 'joy'],
 ['anger', 'fear'],
 ['anger', 'surprise'],
 ['fear', 'surprise'],
 ['anger'],
 ['neutral'],
 ['anger', 'fear'],
 ['anger', 'fear'],
 ['fear', 'surprise'],
 ['fear', 'surprise'],
 ['joy', 'neutral'],
 ['anger', 'fear', 'joy'],
 ['joy', 'surprise'],
 ['anger', 'disgust'],
 ['neutral'],
 ['surprise'],
 ['joy', 'neutral'],
 ['surprise', 'joy'],
 ['neutral'],
 ['fear', 'joy'],
 ['joy'],
 ['joy

In [282]:
for idx, (i,j) in enumerate(zip(grounds, predictions)):
    if len(i) != len(j):
        print(idx, len(i), len(j))

0 1 3
1 1 3
3 1 2
4 2 1
8 1 2
11 1 2
13 1 3
14 1 2
15 1 2
18 1 2
21 1 2
22 1 2
23 2 1
27 1 0
35 2 1
42 3 1
43 2 1
44 2 1
49 2 1
51 2 1
55 1 2
56 1 2
57 2 3
58 3 2
60 1 3
63 1 2
69 1 2
70 1 2
72 1 2
73 1 2
74 1 3
75 1 2
81 2 1
85 2 1
86 2 1
87 2 1
89 2 1
91 1 2
93 2 1
98 1 2
99 2 1
100 1 2
101 1 2
104 1 2
105 2 1
107 1 2
109 1 2
111 1 2
112 1 2
116 2 1
118 1 2
122 2 1
125 1 2
129 2 1
133 1 2
134 2 1
137 2 1
138 1 2
139 1 2
146 2 1
155 2 1
156 1 2
164 1 3
165 1 3
167 1 2
168 1 2
169 2 3
172 1 2
173 2 1
174 3 1
175 2 1
182 1 2
183 1 2
188 2 1
189 2 1
190 2 1
191 2 1
197 2 1
204 2 1
211 1 2
212 1 2
213 1 2
215 1 2
216 1 2
217 1 2
220 2 1
221 1 2
225 1 2
229 2 1
231 2 1
232 2 1
233 2 1
235 2 1
241 3 2
243 2 1
245 1 2
249 2 1
250 2 1
251 2 1
259 1 2
260 2 3
261 2 3
262 1 3
264 1 2
265 1 2
267 2 1
276 2 1
277 1 2
280 2 1
281 2 1
289 1 2
290 2 1
294 2 1
296 2 1
297 3 2
298 3 2
302 2 1
303 2 1
310 2 1
311 2 1
312 2 1
315 2 1
317 2 1
318 2 1
319 2 1
320 3 1
321 2 1
322 1 2
325 1 2
329 1 2
332 1 

In [283]:
# grounds = [item for sublist1 in grounds for sublist2 in sublist1 for item in sublist2]
# predictions = [item for sublist1 in predictions for sublist2 in sublist1 for item in sublist2]

In [342]:
len(grounds), len(predictions)

(3685, 3685)

In [343]:
mlb = MultiLabelBinarizer()

In [344]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['admiration', 'aggression', 'confusion', 'curiosity', 'determination', 'embarrassment', 'encouragement', 'gratitude', 'hope', 'love', 'trust'] will be ignored
  warnings.warn(


In [345]:
y_true_mhot.shape

(3685, 7)

In [346]:
y_true_mhot.shape, y_pred_mhot.shape

((3685, 7), (3685, 7))

In [347]:
class_labels = mlb.classes_

In [348]:
class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [349]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.490     0.632     0.552      1253
     disgust      0.148     0.361     0.210       194
        fear      0.358     0.496     0.416       983
         joy      0.440     0.425     0.432       791
     neutral      0.108     0.597     0.183       221
     sadness      0.562     0.299     0.391       962
    surprise      0.412     0.456     0.433       973

   micro avg      0.363     0.474     0.411      5377
   macro avg      0.360     0.467     0.374      5377
weighted avg      0.429     0.474     0.432      5377
 samples avg      0.371     0.486     0.397      5377



In [313]:
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/saved_models/comics35_pg_old_Meta-Llama-3.1-70B-Instruct-bnb-4bit/classification_report_3.pickle", 'wb') as fh:
    
    pickle.dump(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3, output_dict=True), fh)

In [ ]:
# grounds = results['grounds']
# predictions = results['predictions']

In [ ]:
# grounds = [json.loads(x)["page_utterance_emotions"] for x in grounds]
# predictions = [json.loads(x)["page_utterance_emotions"] for x in predictions]
#predictions = [x['page_utterance_emotions'] for x in predictions]

In [ ]:
#grounds

[[['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral']],
 [['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise']],
 [['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise'],
  ['anger', 

In [ ]:
#grounds_u = [[item for sublist in outer_list for item in sublist] for outer_list in grounds]

In [ ]:
#predictions_u = [[item for sublist in outer_list for item in sublist] for outer_list in predictions]

In [ ]:
#en(grounds_u), len(predictions_u)

(156, 156)

In [ ]:
#grounds_u

[['surprise',
  'joy',
  'joy',
  'surprise',
  'joy',
  'joy',
  'joy',
  'joy',
  'surprise',
  'joy',
  'joy',
  'neutral',
  'neutral'],
 ['neutral',
  'neutral',
  'anger',
  'disgust',
  'anger',
  'disgust',
  'neutral',
  'sadness',
  'sadness'],
 ['anger',
  'sadness',
  'anger',
  'sadness',
  'anger',
  'sadness',
  'fear',
  'surprise',
  'surprise',
  'joy',
  'anger',
  'surprise',
  'joy',
  'joy',
  'joy',
  'anger',
  'anger',
  'surprise',
  'joy',
  'fear',
  'sadness',
  'fear',
  'sadness',
  'fear',
  'surprise'],
 ['anger',
  'disgust',
  'anger',
  'disgust',
  'anger',
  'disgust',
  'fear',
  'sadness',
  'fear',
  'sadness',
  'surprise',
  'sadness',
  'sadness',
  'fear',
  'sadness',
  'sadness',
  'surprise',
  'sadness',
  'surprise',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'disgust',
  'joy',
  'joy',
  'surprise',
  'joy',
  'surprise',
  'joy',
  'anger',
  'surprise',
  'anger',
  'surprise'],
 ['neutral', 'joy', 'joy'],
 ['neutral',
 

In [ ]:
#predictions_u

[['joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy'],
 ['neutral',
  'neutral',
  'anger',
  'surprise',
  'anger',
  'surprise',
  'neutral',
  'anger',
  'disgust',
  'surprise',
  'anger',
  'disgust',
  'surprise'],
 ['anger',
  'anger',
  'sadness',
  'anger',
  'anger',
  'surprise',
  'anger',
  'surprise',
  'sadness',
  'anger',
  'surprise',
  'anger',
  'joy',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'surprise',
  'joy',
  'joy'],
 ['anger',
  'anger',
  'anger',
  'sadness',
  'sadness',
  'sadness',
  'sadness',
  'anger',
  'surprise',
  'surprise',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'joy',
  'anger',
  'anger'],
 ['joy', 'joy', 'joy'],
 ['sadness',
  'fear',
  'surprise',
  'sadness',
  'fear',
  'sadness',
  'joy',
  'sadness',
  'fear',
  'sadness',
  'sadness',
  'joy',
  'sadness',
  'fear',
  'sadness',
  'joy',
  'sadness',
  'joy',
  'joy',
  'sadness',
  'joy',
  'anger',

In [ ]:
#len(grounds_u), len(predictions_u)

(156, 156)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()

In [ ]:
#y_true_mhot = mlb.fit_transform(grounds_u)

In [ ]:
#y_true_mhot.shape

(156, 7)

In [ ]:
#y_true_mhot

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 0, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
#y_pred_mhot = mlb.transform(predictions_u)

In [ ]:
#y_pred_mhot.shape

(156, 7)

In [ ]:
#y_pred_mhot

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
#class_labels = mlb.classes_

In [ ]:
#class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [ ]:
#print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.929     0.778     0.847       117
     disgust      0.318     0.250     0.280        28
        fear      0.756     0.626     0.685        99
         joy      0.855     0.678     0.756        87
     neutral      0.778     0.311     0.444        45
     sadness      0.750     0.734     0.742        94
    surprise      0.948     0.879     0.912       124

   micro avg      0.829     0.692     0.754       594
   macro avg      0.762     0.608     0.667       594
weighted avg      0.825     0.692     0.746       594
 samples avg      0.835     0.692     0.727       594

